In [4]:
import numpy as np
import prettytable as pt

In [5]:
# specify parameters
# первая случайная величина
xs = np.array([4, 5])
# вторая случайная величина
ys = np.array([10, 20])
# совместная функция распределения 2 случаных величин
# (откуда только мы можем ее получить в реальном мире)
f = np.array([
    [0.3, 0.2], 
    [0.1, 0.4]
])

# не очень понятно что это, так как классическое определение 
# joint cumulative fucntion совершенно другое
f_cum = np.cumsum(f)
print(f"f_cum = {f_cum}")
# генерируем случайные числа из uniform distribution
sample_size = 1_000_000
uniform_sample = np.random.rand(sample_size)

# создаем joint случайную величину X
x = np.vstack([
    xs[1] * np.ones(uniform_sample.shape), 
    ys[1] * np.ones(uniform_sample.shape)
])

# map to the bivariate distribution
# f_cum = [0.3 0.5 0.6 1. ]
# приравниваем тем значениям верояность которых  
# меньше 0.6 значение 5
x[0, uniform_sample < f_cum[2]] = xs[1]
# меньше 0.6 значение 10
x[1, uniform_sample < f_cum[2]] = ys[0]

# меньше 0.5 значение 4
x[0, uniform_sample < f_cum[1]] = xs[0]
# меньше 0.5 значение 20
x[1, uniform_sample < f_cum[1]] = ys[1]

# меньше 0.3 значение 4
x[0, uniform_sample < f_cum[0]] = xs[0]
# меньше 0.3 значение 10
x[1, uniform_sample < f_cum[0]] = ys[0]
# получили распределение данных согласно joint distribution
print(x)

f_cum = [0.3 0.5 0.6 1. ]
[[ 5.  5.  5. ...  5.  5.  4.]
 [20. 10. 10. ... 20. 20. 20.]]


In [6]:
# marginal distribution
# вероятность появления 4 в выборке
xp = np.sum(x[0, :] == xs[0]) / sample_size
# вероятность появления 10 в выборке
yp = np.sum(x[1, :] == ys[0]) / sample_size

# print output
print("marginal distribution for x")
xmtb = pt.PrettyTable()
xmtb.field_names = ['x_value', 'x_prob']
xmtb.add_row([xs[0], xp])
# тут мы просто вычели из 1 вероятности, хотя могли по той же
# логике просто посчитать количество тех элементов которые равны 5 
xmtb.add_row([xs[1], 1 - xp])
print(xmtb)

print("\nmarginal distribution for y")
ymtb = pt.PrettyTable()
ymtb.field_names = ['y_value', 'y_prob']
ymtb.add_row([ys[0], yp])
ymtb.add_row([ys[1], 1 - yp])
print(ymtb)

marginal distribution for x
+---------+---------------------+
| x_value |        x_prob       |
+---------+---------------------+
|    4    |       0.500642      |
|    5    | 0.49935799999999997 |
+---------+---------------------+

marginal distribution for y
+---------+--------------------+
| y_value |       y_prob       |
+---------+--------------------+
|    10   |      0.400591      |
|    20   | 0.5994090000000001 |
+---------+--------------------+


In [7]:
print(x)

[[ 5.  5.  5. ...  5.  5.  4.]
 [20. 10. 10. ... 20. 20. 20.]]


In [8]:
# conditional distributions
# выбираем из первой строки только те элементы
# которые равны 10 во второй
xc1 = x[0, x[1, :] == ys[0]]
# которые равны 20 во второй
xc2 = x[0, x[1, :] == ys[1]]
# выбираем из второй строки только те элементы
# которые равны 4 в первой
yc1 = x[1, x[0, :] == xs[0]]
# которые равны 5 в первой
yc2 = x[1, x[0, :] == xs[1]]
# получается что мы обуславливаемся на какие-то значения
print(f"xc1 unique={set(xc1)}")
print(f"yc1 unique={set(yc1)}")

# P(X|Y=10)
xc1p = np.sum(xc1 == xs[0]) / len(xc1)
# P(X|Y=20)
xc2p = np.sum(xc2 == xs[0]) / len(xc2)
# P(Y|X=4)
yc1p = np.sum(yc1 == ys[0]) / len(yc1)
# P(Y|X=5)
yc2p = np.sum(yc2 == ys[0]) / len(yc2)

# print output
print("conditional distribution for x")
xctb = pt.PrettyTable()
xctb.field_names = ['y_value', 'prob(x=4)', 'prob(x=5)']
# вероятность того что х=4, при условии того что мы зафиксировали(получили) у=10 или у=20
# и так как у нас всего 2 значения, мы можем найти второе по формуле обратной вероятности 
xctb.add_row([ys[0], xc1p, 1-xc1p])
# вероятность того что х=5, при условии того что мы зафиксировали(получили) у=10 или у=20
xctb.add_row([ys[1], xc2p, 1-xc2p])
print(xctb)

print("\nconditional distribution for y")
yctb = pt.PrettyTable()
yctb.field_names = ['x_value',  'prob(y=10)', 'prob(y=20)']
yctb.add_row([xs[0], yc1p, 1-yc1p])
yctb.add_row([xs[1], yc2p, 1-yc2p])
print(yctb)

xc1 unique={4.0, 5.0}
yc1 unique={10.0, 20.0}
conditional distribution for x
+---------+--------------------+--------------------+
| y_value |     prob(x=4)      |     prob(x=5)      |
+---------+--------------------+--------------------+
|    10   | 0.7504487120279787 | 0.2495512879720213 |
|    20   | 0.3336936882829587 | 0.6663063117170414 |
+---------+--------------------+--------------------+

conditional distribution for y
+---------+---------------------+---------------------+
| x_value |      prob(y=10)     |      prob(y=20)     |
+---------+---------------------+---------------------+
|    4    |  0.6004749901126953 | 0.39952500988730466 |
|    5    | 0.20019304787346953 |  0.7998069521265305 |
+---------+---------------------+---------------------+


In [9]:
class discrete_bijoint:

    def __init__(self, 
            joint_probbability_matrix, 
            x_values, 
            y_values
    ):
        '''initialization
        -----------------
        parameters:
        f: the bivariate joint probability matrix
        xs: values of x vector
        ys: values of y vector
        '''
        self.joint_probbability_matrix = joint_probbability_matrix
        self.x_values = x_values
        self.y_values = y_values
        self.jtb = None

    def joint_tb(self):
        '''print the joint distribution table'''
        x_values = self.x_values
        y_values = self.y_values
        joint_probbability_matrix = self.joint_probbability_matrix
        
        jtb = pt.PrettyTable()
        jtb.field_names = [
            'x_value/y_value', 
            *y_values, 
            'marginal sum for x'
        ]
        
        for i in range(len(x_values)):
            jtb.add_row([
                x_values[i], 
                *joint_probbability_matrix[i, :], 
                # высчитываем marginal probbability для величины х
                np.sum(joint_probbability_matrix[i, :])
            ])
        
        jtb.add_row(['marginal_sum for y', 
			*np.sum(joint_probbability_matrix, 0), 
   			np.sum(joint_probbability_matrix)
      	])
        print("\nThe joint probability distribution for x and y\n", jtb)
        self.jtb = jtb

    def draw(self, samples_amount):
        '''draw random numbers
        ----------------------
        parameters:
        n: number of random numbers to draw
        '''
        xs = self.x_values
        ys = self.y_values
        # последовательно складываем вероятности в получившейся совместной матрице
        
        f_cum = np.cumsum(self.joint_probbability_matrix)
        p = np.random.rand(samples_amount)
        x = np.empty([2, p.shape[0]])
        cum_sum_len = len(f_cum) - 1
        lx = len(xs)-1
        ly = len(ys)-1
        
        for i in range(cum_sum_len+1):
            x[0, p < f_cum[cum_sum_len-i]] = xs[lx]
            x[1, p < f_cum[cum_sum_len-i]] = ys[ly]
            if ly == 0:
                lx -= 1
                ly = len(ys)-1
            else:
                ly -= 1
        self.x = x
        self.n = samples_amount

    def marg_dist(self):
        '''marginal distribution'''
        x = self.x
        xs = self.x_values
        ys = self.y_values
        n = self.n
        xmp = [np.sum(x[0, :] == xs[i])/n for i in range(len(xs))]
        ymp = [np.sum(x[1, :] == ys[i])/n for i in range(len(ys))]

        # print output
        xmtb = pt.PrettyTable()
        ymtb = pt.PrettyTable()
        xmtb.field_names = ['x_value', 'x_prob']
        ymtb.field_names = ['y_value', 'y_prob']
        for i in range(max(len(xs), len(ys))):
            if i < len(xs):
                xmtb.add_row([xs[i], xmp[i]])
            if i < len(ys):
                ymtb.add_row([ys[i], ymp[i]])
        xmtb.add_row(['sum', np.sum(xmp)])
        ymtb.add_row(['sum', np.sum(ymp)])
        print("\nmarginal distribution for x\n", xmtb)
        print("\nmarginal distribution for y\n", ymtb)

        self.xmp = xmp
        self.ymp = ymp

    def cond_dist(self):
        '''conditional distribution'''
        x = self.x
        xs = self.x_values
        ys = self.y_values
        n = self.n
        xcp = np.empty([len(ys), len(xs)])
        ycp = np.empty([len(xs), len(ys)])
        for i in range(max(len(ys), len(xs))):
            if i < len(ys):
                xi = x[0, x[1, :] == ys[i]]
                idx = xi.reshape(len(xi), 1) == xs.reshape(1, len(xs))
                xcp[i, :] = np.sum(idx, 0)/len(xi)
            if i < len(xs):
                yi = x[1, x[0, :] == xs[i]]
                idy = yi.reshape(len(yi), 1) == ys.reshape(1, len(ys))
                ycp[i, :] = np.sum(idy, 0)/len(yi)

        # print output
        xctb = pt.PrettyTable()
        yctb = pt.PrettyTable()
        xctb.field_names = ['x_value', *xs, 'sum']
        yctb.field_names = ['y_value', *ys, 'sum']
        for i in range(max(len(xs), len(ys))):
            if i < len(ys):
                xctb.add_row([ys[i], *xcp[i], np.sum(xcp[i])])
            if i < len(xs):
                yctb.add_row([xs[i], *ycp[i], np.sum(ycp[i])])
        print("\nconditional distribution for x\n", xctb)
        print("\nconditional distribution for y\n", yctb)

        self.xcp = xcp
        self.xyp = ycp

In [10]:
# joint
d = discrete_bijoint(f, xs, ys)
d.joint_tb()


The joint probability distribution for x and y
 +--------------------+-----+--------------------+--------------------+
|  x_value/y_value   |  10 |         20         | marginal sum for x |
+--------------------+-----+--------------------+--------------------+
|         4          | 0.3 |        0.2         |        0.5         |
|         5          | 0.1 |        0.4         |        0.5         |
| marginal_sum for y | 0.4 | 0.6000000000000001 |        1.0         |
+--------------------+-----+--------------------+--------------------+
